# Proyecto - Deteccion y mitigacion de ataques DDoS en redes IoT/Cloud usando ML/DL

## Paso 2. Entrenamiento de modelos

Una vez tenemos los datos preprocesados y listos para usarse, se procede con
el entrenamiento de los modelos de Machine y Deep learning propuestos.

---

### Importar librerias necesarias

In [ ]:
import pandas as pd
import numpy as np

from ddosmllib.data import get_dataset_df, DatasetProyecto
from dotenv import load_dotenv
from os.path import join as pjoin
from os import getenv

from joblib import dump as model_save
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (classification_report, confusion_matrix,
                            roc_auc_score, f1_score, make_scorer)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.utils import to_categorical

# Cargamos la variable de entorno para guardar los modelos entrenados. Si no
# existe, se lanza una excepción.
load_dotenv('../.env')
if not getenv('TRAINED_MODELS_PATH'):
    raise RuntimeError('Environment variable TRAINED_MODELS_PATH not defined')

rnd_state: int = 36  # Se define semilla para reproducibilidad

### Entrenamiento con CIC DDoS 2019

Primero cargamos los datos y establecemos las características y etiqueta.
Luego, dividimos los datos en conjuntos de entrenamiento y prueba usando la
funcion `train_test_split` de `sklearn`.

In [ ]:
# Obtenemos el DataFrame del dataset CIC DDoS 2019
df = get_dataset_df(DatasetProyecto.CIC_DDoS2019)

# Definimos las características y la etiqueta
X = df.drop(['Label_encoded'], axis=1)
y = df['Label_encoded']

# Subdividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        random_state=rnd_state,
        stratify=y
)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
# Se guardan los datos de testing en un archivo
df_test = pd.DataFrame(X_test)
df_test['Label_encoded'] = y_test
df_test.to_parquet(pjoin(getenv('TRAINED_MODELS_PATH'), 'CICDDOS2019-test.parquet'))

#### Random Forest

Primero cargamos los datos y establecemos las características y etiqueta.
Luego, dividimos los datos en conjuntos de entrenamiento y prueba usando la
funcion `train_test_split` de `sklearn`. Posteriormente, creamos un pipeline
para aplicar la tecnica de sobremuestreo SMOTE y entrenamos un modelo de
Random Forest. La tecnica es utilzada para balancear las clases del conjunto,
 ya que el conjunto de datos CIC DDoS 2019 posee muchas más instancias de
 datos malignos que benignos.

In [ ]:
clf = RandomForestClassifier(
        n_estimators=100,
        random_state=rnd_state,
        class_weight='balanced'
)

smote = SMOTE(random_state=rnd_state)

pipeline = ImbPipeline([
    ('smote', smote),
    ('classifier', clf)
])

# Optimización de hiperparámetros
param_grid = {
    'classifier__n_estimators': [10, 100],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__bootstrap': [True]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=3,
        n_jobs=-1,
        verbose=2,
        scoring='f1_macro',
)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [ ]:
# Evaluar el mejor modelo
best_clf = grid_search.best_estimator_
y_pred_best = best_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

In [ ]:
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)
y_proba = best_clf.predict_proba(X_test)

In [ ]:
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

print("ROC AUC:", roc_auc_score(y_test, y_proba))
print("F1-Score:", f1_score(y_test, y_pred))

Y con las métricas de rendimiento, se guarda el modelo para su posterior uso
en el backend

In [ ]:
# Se guarda el modelo en un archivo
model_save(pipeline, pjoin(getenv('TRAINED_MODELS_PATH'), 'CICDDOS2019-RF.joblib'))

#### LSTM

Primero se realiza una transformación de los datos para que puedan ser
utilizados en el modelo LSTM. Luego, se define el modelo y se entrena, para
guardarlo posteriormente y mostrar las métricas de rendimiento.

In [ ]:
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_train.shape[0], 1, X_train.shape[1]))

num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)


In [ ]:
# Se define el modelo LSTM
model_cic = Sequential()
model_cic.add(LSTM(64, input_shape=(
    X_train_lstm.shape[1], X_train_lstm.shape[2])))
model_cic.add(Dense(num_classes, activation='softmax'))

# Se compila el modelo
model_cic.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

# Se entrena el modelo
print("\nEntrenando modelo LSTM para CIC-DDoS2019...")
model_cic.fit(X_train_lstm, y_train_cat, epochs=5, batch_size=64,
              validation_data=(X_test_lstm, y_test_cat))

model_cic.save(pjoin(getenv('TRAINED_MODELS_PATH'), 'CICDDOS2019-LSTM.h5'))

Con el modelo entrenado, se obtienen las métricas de rendimiento.

In [ ]:
# Evaluación para CIC-DDoS2019
print("\nEvaluando modelo LSTM para CIC-DDoS2019...")
y_pred_lstm = model_cic.predict(X_test_lstm)
y_pred_lstm_classes = np.argmax(y_pred_lstm, axis=1)

print("\nReporte de clasificación para LSTM en CIC-DDoS2019:")
print(classification_report(y_test, y_pred_lstm_classes))

### Entrenamiento con N-BaIoT

Primero cargamos los datos y establecemos las características y etiqueta.
Luego, dividimos los datos en conjuntos de entrenamiento y prueba usando la
funcion `train_test_split` de `sklearn`.

In [ ]:
# Obtenemos el DataFrame del dataset N-BaIoT. Obtenemos un sample de 200000
df = get_dataset_df(DatasetProyecto.N_BaIoT_XGB)
df = df.sample(200000, random_state=rnd_state)

# Definimos las características y la etiqueta
X = df.drop(['malign'], axis=1)
y = df['malign']

# Subdividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        random_state=rnd_state,
        stratify=y
)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
# Se guardan los datos de testing en un archivo
df_test = pd.DataFrame(X_test)
df_test['malign'] = y_test
df_test.to_parquet(pjoin(getenv('TRAINED_MODELS_PATH'), 'NBaIoT-test.parquet'))

#### Random Forest

Primero cargamos los datos y establecemos las características y etiqueta.
Luego, dividimos los datos en conjuntos de entrenamiento y prueba usando la
funcion `train_test_split` de `sklearn`. Posteriormente, creamos un pipeline
para aplicar la tecnica de sobremuestreo SMOTE y entrenamos un modelo de
Random Forest. La tecnica es utilzada para balancear las clases del conjunto,
 ya que el conjunto de datos CIC DDoS 2019 posee muchas más instancias de
 datos malignos que benignos.

In [ ]:
clf = RandomForestClassifier(
        n_estimators=100,
        random_state=rnd_state,
        class_weight='balanced'
)

smote = SMOTE(random_state=rnd_state)

pipeline = ImbPipeline([
    ('smote', smote),
    ('classifier', clf)
])

# Optimización de hiperparámetros
param_grid = {
    'classifier__n_estimators': [10, 100],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__bootstrap': [True]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=3,
        n_jobs=-1,
        verbose=2,
        scoring='f1_macro',
)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [ ]:
# Evaluar el mejor modelo
best_clf = grid_search.best_estimator_
y_pred_best = best_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

In [ ]:
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)
y_proba = best_clf.predict_proba(X_test)

In [ ]:
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

print("ROC AUC:", roc_auc_score(y_test, y_proba))
print("F1-Score:", f1_score(y_test, y_pred))

Y con las métricas de rendimiento, se guarda el modelo para su posterior uso
en el backend

In [ ]:
# Se guarda el modelo en un archivo
model_save(pipeline, pjoin(getenv('TRAINED_MODELS_PATH'), 'NBAIOT-RF.joblib'))

#### LSTM

Primero se realiza una transformación de los datos para que puedan ser
utilizados en el modelo LSTM. Luego, se define el modelo y se entrena, para
guardarlo posteriormente y mostrar las métricas de rendimiento.

In [ ]:
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_train.shape[0], 1, X_train.shape[1]))

num_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)


In [ ]:
# Se define el modelo LSTM
model_nbaiot = Sequential()
model_nbaiot.add(LSTM(64, input_shape=(
    X_train_lstm.shape[1], X_train_lstm.shape[2])))
model_nbaiot.add(Dense(num_classes, activation='softmax'))

# Se compila el modelo
model_nbaiot.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

# Se entrena el modelo
print("\nEntrenando modelo LSTM para N-BaIoT...")
model_nbaiot.fit(X_train_lstm, y_train_cat, epochs=5, batch_size=64,
              validation_data=(X_test_lstm, y_test_cat))

model_nbaiot.save(pjoin(getenv('TRAINED_MODELS_PATH'), 'NBAIOT-LSTM.h5'))

Con el modelo entrenado, se obtienen las métricas de rendimiento.

In [ ]:
# Evaluación para N-BaIoT
print("\nEvaluando modelo LSTM para N-BaIoT...")
y_pred_lstm = model_nbaiot.predict(X_test_lstm)
y_pred_lstm_classes = np.argmax(y_pred_lstm, axis=1)

print("\nReporte de clasificación para LSTM en N-BaIoT:")
print(classification_report(y_test, y_pred_lstm_classes))